In [1]:
import gc
import pickle
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib.backends.backend_pdf import PdfPages


from boltzmann_machine import BoltzmannMachine


from utils import (plot_sigmoid,
                   get_encprob_env_states, 
                   get_encprob_weight_mask, 
                   gen_free_state_dist_pdf, 
                   gen_clamped_state_dist_pdf, 
                   gen_learning_plots_pdf,
                   get_boltzmann_distribution,
                   get_probs_from_counts,
                   plot_series,
                   stringify_vec)

grp_size = 4
num_hnodes = 2
env_states = get_encprob_env_states(grp_size)
weight_mask = get_encprob_weight_mask(grp_size, num_hnodes) # two vis grps should not interact with each other and hidden units should not interact with each other

In [2]:
def get_encNetwork_scores(bm):
    uniqs_len_list = []
    for clamped_debug_list in (bm.clamped_run_debug):
        env_hidd_states = []
        for clamped_dist in clamped_debug_list:
            max_count_hidd_state = bm.clamped_run_eval(clamped_dist, print_eval=False)
            env_hidd_states.append(stringify_vec(max_count_hidd_state))

        uniqs_len_list.append(len(np.unique(env_hidd_states)))

    return uniqs_len_list

In [3]:
bm_list = []

noise_multiplicity = 10
learning_cycles = 1000

for i in range(10):
    bm = BoltzmannMachine(env_states, 
                        num_hnodes,
                        weight_mask=weight_mask, 
                        noisy_clamping=True,
                        logging=False,
                        num_noisyEnvState_per_envState=noise_multiplicity)
    bm.learn(learning_cycles)

    last_clamped_debug_list = bm.clamped_run_debug[-1]
    last_free_run_dist = bm.free_run_debug[-1]


    print(f'#{i+1}', end=' ')
    for clamped_run_dist in last_clamped_debug_list:
        bm.clamped_run_eval(clamped_run_dist)
    bm.free_run_eval(last_free_run_dist)

    bm_list.append(bm)

  0%|          | 0/1000 [00:00<?, ?it/s]

#1 [0. 0.], [0. 1.], [1. 0.], [1. 1.], [0, 1, 2, 3, 113]


  0%|          | 0/1000 [00:00<?, ?it/s]

#2 [1. 0.], [1. 1.], [0. 1.], [0. 0.], [0, 1, 2, 3, 76, 77]


  0%|          | 0/1000 [00:00<?, ?it/s]

...

#8 [0. 1.], [1. 0.], [1. 1.], [0. 0.], [0, 1, 2, 3, 73, 88, 117, 137]


  0%|          | 0/1000 [00:00<?, ?it/s]

#9 [1. 1.], [1. 0.], [0. 1.], [0. 0.], [0, 1, 2, 3, 100, 109, 116]


  0%|          | 0/1000 [00:00<?, ?it/s]

#10 [1. 1.], [0. 1.], [0. 0.], [1. 0.], [0, 1, 2, 3, 99, 103, 143, 151]


In [4]:
for i, bm in enumerate(bm_list):
    scores = get_encNetwork_scores(bm)

    print(f'#{i+1}', end=' ')
    print(scores[-5:], end='; ')

    bm.free_run_eval(bm.free_run_debug[-1])

#1 [4, 4, 4, 4, 4]; [0, 1, 2, 3, 113]
#2 [4, 4, 4, 4, 4]; [0, 1, 2, 3, 76, 77]
#3 [4, 4, 4, 4, 4]; [0, 1, 2, 3, 93, 95, 129, 136]
#4 [4, 4, 4, 4, 4]; [0, 1, 2, 3, 79, 111, 115]
#5 [4, 4, 4, 4, 4]; [0, 1, 2, 3, 106, 110]
#6 [3, 3, 3, 3, 3]; [0, 1, 2, 5, 27, 36, 66]
#7 [4, 4, 4, 4, 4]; [0, 1, 2, 3, 73, 115, 120]
#8 [4, 4, 4, 4, 4]; [0, 1, 2, 3, 73, 88, 117, 137]
#9 [4, 4, 4, 4, 4]; [0, 1, 2, 3, 100, 109, 116]
#10 [4, 4, 4, 4, 4]; [0, 1, 2, 3, 99, 103, 143, 151]


In [ ]:
# init_W = None
# bm_idx = 6

# bm2 = BoltzmannMachine(env_states, 
#             num_hnodes,
#             weight_mask=weight_mask, 
#             noisy_clamping=True,
#             init_W=bm_list[bm_idx].learning_debug[-1]['W'],
#             # init_W=init_W,
#             logging=True)

# bm2.learn(50)

  0%|          | 0/50 [00:00<?, ?it/s]

[0. 0.], [0. 1.], [1. 0.], [0. 1.], [0, 1, 14, 23, 28, 42, 54, 61, 68, 82, 91]
[0. 0.], [0. 1.], [1. 0.], [1. 1.], [0, 1, 2, 6, 9, 11, 48, 61, 66, 83, 87, 325]
[0. 0.], [0. 1.], [1. 0.], [0. 1.], [0, 1, 12, 21, 26, 41, 53, 58, 64, 74, 82]
[0. 0.], [0. 1.], [1. 0.], [1. 1.], [0, 1, 2, 7, 10, 14, 48, 63, 68, 82, 88, 315]
[0. 0.], [0. 1.], [1. 0.], [1. 1.], [0, 1, 5, 7, 18, 22, 56, 60, 73, 84, 91, 283]
[0. 0.], [0. 1.], [1. 0.], [1. 1.], [0, 1, 2, 5, 7, 10, 53, 66, 71, 90, 97]
[0. 0.], [0. 1.], [1. 0.], [1. 1.], [0, 1, 6, 10, 22, 29, 61, 67, 79, 94, 114, 292]
[0. 0.], [0. 1.], [1. 0.], [1. 1.], [0, 3, 4, 5, 6, 12, 58, 63, 65, 96, 100, 345]
[0. 0.], [0. 1.], [1. 0.], [1. 1.], [0, 1, 6, 10, 23, 30, 61, 68, 80, 99, 118, 296]
[0. 0.], [1. 1.], [1. 0.], [1. 1.], [0, 3, 4, 5, 6, 12, 58, 63, 70, 99, 104]
[0. 0.], [1. 1.], [1. 0.], [1. 1.], [0, 1, 6, 16, 17, 30, 68, 71, 80, 81, 120, 292]
[0. 0.], [0. 1.], [1. 0.], [1. 1.], [0, 2, 3, 4, 6, 10, 56, 61, 71, 98, 102, 272, 364]
[0. 0.], [0. 1.], [1. 0

In [11]:
for free_run_dist in bm_list[0].free_run_debug:
    bm.free_run_eval(free_run_dist)

[43, 153]
[21, 27, 43, 44, 55, 67, 109, 129, 141]
[1, 24, 78, 79, 94, 106, 138, 139]
[16, 26, 27, 37, 49, 65, 116, 139, 151]
[14, 24, 39, 46, 57, 84]
[3, 7, 30, 40, 56, 60, 79]
[6, 40, 49, 58, 59, 73, 74, 101]
[8, 21, 26, 57, 68, 122]
[12, 18, 19, 24, 42, 55, 103]
[22, 26, 27, 37, 40, 46, 66, 68, 116]
[2, 8, 29, 31, 42, 44, 45, 48, 58]
[8, 15, 16, 22, 40, 41, 44, 58, 59, 75]
[11, 26, 31, 32, 49, 52, 63, 68, 83, 99]
[1, 13, 14, 19, 31, 43, 47, 56, 92]
[0, 15, 17, 21, 38, 40, 43, 55, 56, 66, 76, 101]
[13, 20, 28, 30, 31, 38, 49, 50, 61, 81]
[0, 16, 24, 34, 38, 39, 41, 44, 45, 57, 82]
[2, 8, 11, 17, 32, 34, 42, 47, 51]
[0, 10, 18, 25, 26, 32, 52, 64, 65, 73, 90]
[5, 8, 14, 18, 19, 21, 32, 36, 53, 73]
[0, 11, 25, 30, 31, 34, 47]
[2, 5, 9, 14, 20, 23, 33, 47]
[2, 10, 11, 22, 23, 28, 29, 40, 41, 42, 51]
[1, 4, 8, 9, 19, 21, 25, 30, 34, 47]
[0, 1, 11, 13, 21, 27, 31, 41, 44]
[4, 8, 9, 19, 23, 48, 49, 59, 63, 77]
[4, 5, 6, 15, 27, 28, 32, 36, 38, 39, 50, 62, 66]
[1, 2, 5, 23, 27, 33, 40, 41, 5

#### Result
- First run - 4/10 
- 3/10
- 3/10
- 4/10; lower noise multiplocity of 100
- 3/10;  lower noise multiplocity of 50
- 3/10;  lower noise multiplocity of 50
- 4/10; noisy_multiplicty=30, learning_cycles=50*2
- 2/10; noisy_multiplicty=30, learning_cycles=50*2
- 5/5; noisy_multiplicty=20, learning_cycles=500*2 
- 5/5; noisy_multiplicty=20, learning_cycles=500*2 
- 6/10; noisy_multiplicty=10, learning_cycles=1000
- 7/10; noisy_multiplicty=10, learning_cycles=1000
- 7/10; noisy_multiplicty=""1"", learning_cycles=2000
- 9/10; noisy_multiplicty=10, learning_cycles=1000
- 9/10; noisy_multiplicty=10, learning_cycles=1000

In [15]:
# base_dir = './saved_run/nc_signed_fixedFreeRun_T10_t20/'

# with open(f'{base_dir}/bm_object_free_run_debug.pkl', 'wb') as outp:
#     pickle.dump(bm.free_run_debug, outp, pickle.HIGHEST_PROTOCOL)

# with open(f'{base_dir}/bm_object_clamped_run_debug.pkl', 'wb') as outp:
#     pickle.dump(bm.clamped_run_debug, outp, pickle.HIGHEST_PROTOCOL)

# with open(f'{base_dir}/bm_object_learning_debug.pkl', 'wb') as outp:
#     pickle.dump(bm.learning_debug, outp, pickle.HIGHEST_PROTOCOL)